In [168]:
from ib.ext.Contract import Contract
from ib.ext.Order import Order
from ib.opt import Connection, message
import ib.opt
import ib.ext.Contract
from ib.ext.TickType import TickType as tt
 
import time
import pandas as pd

import sys

if sys.version_info.major == 2:
    import Queue as queue
else:  # >= 3
    import queue

In [355]:
class IbManager(object):
    def __init__(self, timeout=20, **kwargs):
        self.q = queue.Queue()
        self.timeout = 5

        self.con = ibConnection(**kwargs)
        
        #self.con.registerAll(self.watcher)
        
        self.contracts = pd.DataFrame([
                ['IBM','SMART','USD'],
                ['AAPL','SMART','USD'],
                ['GOOG','SMART','USD']
        ],columns = ['sym','exch','curr'])

        #add these specific column names to match the name returned by TickType.getField()
        self.contracts['bidPrice'] = 0
        self.contracts['askPrice'] = 0


        self.msgs = {
            ib.opt.message.error: self.errors,
            ib.opt.message.contractDetails: self.contractDetailsHandler,
            ib.opt.message.contractDetailsEnd: self.contractDetailsHandler,
            ib.opt.message.tickPrice: self.tickPriceHandler
        }

        self.skipmsgs = tuple(self.msgs.keys())

        for msgtype, handler in self.msgs.items():
            self.con.register(handler, msgtype)

        self.con.connect()
        sleep(1)
        self.con.reqMarketDataType( 3 )
        
    def tickPriceHandler(self, msg):
        print (msg,msg.tickerId,msg.field,msg.price)
        if msg.field in [66,67]:
            if msg.field == 66:
                field = 'bidPrice'
            if msg.field == 67:
                field = 'askPrice'   
            self.contracts.loc[msg.tickerId,field] = msg.price

    def watcher(self, msg):
        if isinstance(msg, ib.opt.message.error):
            if msg.errorCode > 2000:  # informative message
                print('-' * 10, msg)

        elif not isinstance(msg, self.skipmsgs):
            print('-' * 10, msg)

    def errors(self, msg):
        if msg.id is None:  # something is very wrong in the connection to tws
            self.q.put((True, -1, 'Lost Connection to TWS'))
        elif msg.errorCode < 1000:
            self.q.put((True, msg.errorCode, msg.errorMsg))

    def contractDetailsHandler(self, msg):
        if isinstance(msg, ib.opt.message.contractDetailsEnd):
            self.q.put((False, msg.reqId, msg))
        else:
            self.q.put((False, msg.reqId, msg.contractDetails))

    def get_contract_details(self, contract):
        self.con.reqContractDetails(1, contract)

        cdetails = list()
        df = pd.DataFrame()

        while True:
            try:
                err, mid, msg = self.q.get(block=True, timeout=self.timeout)
            except queue.Empty:
                err, mid, msg = True, -1, "Timeout receiving information"
                break

            if isinstance(msg, ib.opt.message.contractDetailsEnd):
                mid, msg = None, None
                break

            cdetails.append(msg)  # must be contractDetails
            
            if df.shape[0]==0:
                df = pd.DataFrame(columns=list(msg.m_summary.__dict__.keys()))
                df.loc[len(df)] = pd.Series(msg.m_summary.__dict__)
            else:
                df.loc[len(df)] = pd.Series(msg.m_summary.__dict__)

        return df, err, mid, msg

In [356]:

def esFutContract():
    contract = Contract()
    contract.m_symbol = "ES"
    contract.m_exchange = "GLOBEX"
    contract.m_secType = "FUT"
    contract.m_tradingClass = 'ES'
    
    return contract

def esOptContract(exp):
    contract = Contract()
    contract.m_symbol = "ES"
    contract.m_secType = "FOP"
    contract.m_exchange = "GLOBEX"
    #contract.m_right = "C"
    #contract.m_localSymbol = spFuts[0].m_localSymbol
    contract.m_expiry = exp
 
    return contract

In [366]:
ibm.con.disconnect()
#ibm = IbManager("127.0.0.1", port=7497, clientId=998)


True

In [358]:
def vxOptContract(exp):
    contract = Contract()
    contract.m_symbol = "VIX"
    contract.m_secType = "OPT"
    contract.m_exchange = "SMART"
    #contract.m_right = "C"
    #contract.m_localSymbol = spFuts[0].m_localSymbol
    contract.m_expiry = exp
    contract.m_tradingClass='VIX'
 
    return contract

def spOptContract(exp):
    contract = Contract()
    contract.m_symbol = "SPX"
    contract.m_secType = "OPT"
    contract.m_exchange = "SMART"
    #contract.m_right = "C"
    #contract.m_localSymbol = spFuts[0].m_localSymbol
    contract.m_expiry = exp
    contract.m_tradingClass='SP'
 
    return contract

def vixFutContract(localSym=None):
    contract = Contract()
    contract.m_symbol = "VIX"
    contract.m_exchange = "CFE"
    contract.m_secType = "FUT"
    contract.m_tradingClass = 'VX'
    contract.m_localSymbol = localSym

    return contract

def spFutContract(exp=None):
    contract = Contract()
    contract.m_symbol = "SPX"
    contract.m_exchange = "GLOBEX"
    contract.m_secType = "FUT"
    contract.m_tradingClass = 'SP'
    contract.m_expiry = exp
    
    return contract

In [359]:
cdetails, err, errid, errmsg = ibm.get_contract_details(vixFutContract('VXZ8'))
#cdetails, err, errid, errmsg = ibm.get_contract_details(esFutContract())
#cdetails, err, errid, errmsg = ibm.get_contract_details(esOptContract('20190315'))
#cdetails, err, errid, errmsg = ibm.get_contract_details(vxOptContract('20190115'))
#cdetails, err, errid, errmsg = ibm.get_contract_details(spOptContract('20190118'))
#cdetails, err, errid, errmsg = ibm.get_contract_details(spFutContract('20190314'))

In [360]:
cdetails

,m_conId,m_strike,m_includeExpired,m_symbol,m_secType,m_expiry,m_right,m_exchange,m_currency,m_localSymbol,m_tradingClass,m_multiplier,m_primaryExch
0,312051008,0.0,False,VIX,FUT,20181219,None,CFE,USD,VXZ8,VX,1000,None


In [361]:
for index, row in ibm.contracts.iterrows():
    c = Contract()
    c.m_symbol = row['sym']
    c.m_exchange = row['exch']
    c.m_currency = row['curr']
    c.m_secType = 'STK' 
    ibm.con.reqMktData(index, c, '', False)
    #break

<tickPrice tickerId=1, field=66, price=165.64, canAutoExecute=-1> 1 66 165.64
<tickPrice tickerId=1, field=67, price=165.95, canAutoExecute=-1> 1 67 165.95
<tickPrice tickerId=1, field=68, price=165.84, canAutoExecute=0> 1 68 165.84
<tickPrice tickerId=1, field=72, price=167.53, canAutoExecute=0> 1 72 167.53
<tickPrice tickerId=1, field=73, price=164.39, canAutoExecute=0> 1 73 164.39
<tickPrice tickerId=1, field=75, price=163.94, canAutoExecute=0> 1 75 163.94
<tickPrice tickerId=1, field=76, price=0.0, canAutoExecute=0> 1 76 0.0
<tickPrice tickerId=2, field=66, price=1021.17, canAutoExecute=-1> 2 66 1021.17
<tickPrice tickerId=2, field=67, price=1028.11, canAutoExecute=-1> 2 67 1028.11
<tickPrice tickerId=2, field=68, price=1026.0, canAutoExecute=0> 2 68 1026.0
<tickPrice tickerId=2, field=72, price=1049.48, canAutoExecute=0> 2 72 1049.48
<tickPrice tickerId=2, field=73, price=1021.44, canAutoExecute=0> 2 73 1021.44
<tickPrice tickerId=2, field=75, price=1016.53, canAutoExecute=0> 2 75

In [365]:
ibm.contracts

<tickPrice tickerId=0, field=67, price=116.88, canAutoExecute=-1> 0 67 116.88


,sym,exch,curr,bidPrice,askPrice
0,IBM,SMART,USD,116.02,116.88
1,AAPL,SMART,USD,165.55,165.70
2,GOOG,SMART,USD,1021.17,1028.07


<tickPrice tickerId=0, field=66, price=116.01, canAutoExecute=-1> 0 66 116.01
